In [27]:
import numpy as np
import igl
import meshplot as mp

In [28]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [29]:
pi, v = igl.read_triangle_mesh("data/luigi.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

# Setting up  the constraints

In [30]:
class SpatialIndex:
    # Build the grid
    def __init__(self, points, cell_size):
        self.points = points
        self.cell_size = cell_size
        self.grid = {}

        for idx, point in enumerate(points):
            cell = self._get_corresponding_cell(point)
            if cell not in self.grid:
                self.grid[cell] = []
            self.grid[cell].append(idx)

    # Get cell corresponding to a point
    def _get_corresponding_cell(self, point):
        return tuple(np.floor(point / self.cell_size).astype(int))

    # Get neighboring cells within a given radius
    def _get_neighboring_cells(self, point, radius):
        min_corner = point - radius
        max_corner = point + radius

        min_cell = self._get_corresponding_cell(min_corner)
        max_cell = self._get_corresponding_cell(max_corner)

        ranges = [range(min_cell[i], max_cell[i] + 1) for i in range(3)]
        return [tuple(c) for c in np.array(np.meshgrid(*ranges, indexing="ij")).reshape(3, -1).T]

    # Find the closest point in the grid
    def find_closest_point(self, point):
        best_dist = float('inf')
        best_idx = -1
        search_radius = self.cell_size
        found = False

        while not found:
            for cell in self._get_neighboring_cells(point, search_radius):
                if cell in self.grid:
                    for idx in self.grid[cell]:
                        d = np.linalg.norm(self.points[idx] - point)
                        if d < best_dist:
                            best_dist = d
                            best_idx = idx
                            found = True
            # If not found, try again 
            # in a larger radius
            search_radius *= 2
        return best_idx
    
    # Find points within a given radius
    def closest_points(self, point, radius):
        neighbors = []
        seen = set()
        for cell in self._get_neighboring_cells(point, radius):
            if cell in self.grid:
                for idx in self.grid[cell]:
                    if idx in seen: 
                        continue  # Avoid duplicate checks
                    seen.add(idx)
                    if np.linalg.norm(self.points[idx] - point) <= radius + 1e-8:
                        neighbors.append(idx)
        return neighbors


In [31]:
# Add here the code to generate the additional points and constraints

# Function to find the closest point to as point in a set of points
def find_closest_point(point, points):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argmin(distances)

# Function to build constraint equations
def build_constraints(points, normals, epsilon_factor=0.01):
    # Compute the bounding box diagonal
    bbox_min  = np.min(points, axis=0)
    bbox_max  = np.max(points, axis=0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    # Compute epsilon
    epsilon = epsilon_factor * bbox_diag

    # Spatial Index
    grid_resolution = 5
    grid = SpatialIndex(points, grid_resolution)

    p = np.zeros((3*len(points), 3))
    f = np.zeros(3*len(points))

    for i, pi in enumerate(points):
        # Add constraint f(pi) = 0
        p[3 * i] = pi
        f[3 * i] = 0.0
        # Add pi+N/pi+2N's constraints
        for j, sign in enumerate([1, -1]):
            epsilon_current = epsilon
            while True:
                # Compute pi+N/pi+2N
                pi_N = pi + (sign * epsilon_current * normals[i])

                # Find the closest point to pi+N/pi+2N
                # If the point is the same, break
                closest_index = grid.find_closest_point(pi_N)
                debug_1 = find_closest_point(pi_N, points)
                
                assert (debug_1 == closest_index)

                if np.array_equal(points[closest_index], pi):
                    break
                # Otherwise, update epsilon and repeat
                epsilon_current /= 2
            # Append new point and constraints
            p[(3 * i) + j + 1] = pi_N
            f[(3 * i) + j + 1] = sign * epsilon
    
    p = np.array(p)
    f = np.array(f)
    
    return p, f

In [32]:
# Generate the new points and constraints
p, f = build_constraints(pi, ni)

In [33]:
# Create color vector
colors = np.zeros((p.shape[0], 3))
colors[0::3] = [0, 0, 1]
colors[1::3] = [1, 0, 0]
colors[2::3] = [0, 1, 0]
# Visualize the new point cloud 
mp.plot(p, c=colors, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1301440…

# MLS function

In [34]:
def tet_grid_improved(n, points, pca_fix=False):
    # Copy for usage with pca_fix
    points_tmp = points.copy()

    if pca_fix:
        # Center the points
        centroid = np.mean(points_tmp, axis=0)
        centered = points_tmp - centroid

        # Compute eigenvectors
        cov = np.cov(centered.T)
        eigvals, eigvecs = np.linalg.eigh(cov) 

        # Sort eigenvectors
        sort_idx = np.argsort(eigvals)[::-1]
        eigvecs = eigvecs[:, sort_idx]

        # Rotate the points
        points_tmp = centered @ eigvecs

    # Compute the bounding box
    bbox_min = np.min(points_tmp, axis=0)
    bbox_max = np.max(points_tmp, axis=0)

    # Enlarge slightly the bounding box
    bbox_min -= 0.05 * (bbox_max - bbox_min)
    bbox_max += 0.05 * (bbox_max - bbox_min)

    # Generate the grid
    V, T = tet_grid(n, bbox_min, bbox_max)

    if pca_fix:
        # Return to original space
        V = V @ eigvecs.T + centroid

    return V, T

In [35]:
# Resolution
n = 40
resolution = (n, n, n)

# Generate grid n x n x n
x, T = tet_grid_improved(resolution, p, True)

In [36]:
# Function that retrieves the indices of all points in points 
# that are at distance less than h from point.
def closest_points(point, points, h):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argwhere(distances < h).flatten()

def wendland_weight(p, p_i, radius):
    # normalized distance
    r = np.linalg.norm(p_i - p, axis=1)
    q = r / radius
    return (1 - q) ** 4 * (4 * q + 1) * (q < 1)

# Return the polynomial basis functions
# depending on the degree required
def pol_basis(x, degree):
    if degree == 0:
        return np.array([1])
    elif degree == 1:
        return np.array([1, x[0], x[1], x[2]])
    elif degree == 2:
        return np.array([1, x[0], x[1], x[2], x[0]**2, x[1]**2, x[2]**2, 
                         x[0]*x[1], x[1]*x[2], x[2]*x[0]])
    else:
        raise ValueError("Degree must be 0, 1, or 2")

# Evaluate MLS for a given point
def evaluate_MLS(point, points, constraints, radius, k, grid=None):
    # Check if the grid is provided
    # if grid is None:
    neighbors = closest_points(point, points, radius)
    # else:
        #neighbors_brute = closest_points(point, points, radius)
    #neighbors = grid.closest_points(point, radius)
        #assert set(neighbors) == set(neighbors_brute)
    
    # If the number of constraint points is less than twice 
    # the number of polynomial coefficients, return a large positive value
    if len(neighbors) < ((3/2) * k**2 + (3/2) * k + 1):
        return 10e6 
    
    # Linear system components:
    # B is the polynomial basis evaluated at the neighbors
    B = np.array([pol_basis(points[i], k) for i in neighbors])
    # W is the Wendland weight matrix
    W = np.diag(wendland_weight(point, points[neighbors], radius))

    A = B.T @ W @ B
    b = B.T @ W @ constraints[neighbors]
    
    # Solve the linear system
    coeffs = np.linalg.solve(A, b)
    return pol_basis(point, k) @ coeffs

def evaluate_MLS_on_grid(x, points, constraints, h, polyDegree):
    # Build the spatial index
    #grid = SpatialIndex(points, 5)

    fx = np.array([evaluate_MLS(xi, points, constraints, h, polyDegree) for xi in x])
    return fx

In [37]:
# Parameters
wendlandRadius = 1
polyDegree = 2

fx = evaluate_MLS_on_grid(x, p, f, wendlandRadius, polyDegree)

In [38]:
# Assign colors
grid_colors = np.zeros((len(x), 3))
grid_colors[fx >= 0] = [1, 0, 0]  # Red = outside
grid_colors[fx <  0] = [0, 1, 0]  # Green = inside

# Visualize the grid
mp.plot(x, c=grid_colors, shading={"point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.4454464…

---

# Marching to extract surface

In [39]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)

# Plot noisy surface
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.0427577…

In [40]:
# Let's filter out smaller, noisy components

# Count number of faces per component
C = igl.face_components(sf)
component_sizes = np.bincount(C)

# Filter out components with less than min_faces
min_faces = 20000
sf_filtered = sf[component_sizes[C] >= min_faces]

# Keep only used vertices
used_vertices = np.unique(sf_filtered)
# Map from old to new vertex ids
reindex = -np.ones(sv.shape[0], dtype=int)
reindex[used_vertices] = np.arange(len(used_vertices))
# Remaining vertices and faces
sv_filtered = sv[used_vertices]
sf_filtered = reindex[sf_filtered]

# Plot the cleaned surface
mp.plot(sv_filtered, sf_filtered, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1304398…

In [41]:

class SpatialHashGrid:
    def __init__(self, points, cell_size):
        self.points = points
        self.cell_size = cell_size
        self.grid = {}

        for idx, point in enumerate(points):
            cell = self._point_to_cell(point)
            if cell not in self.grid:
                self.grid[cell] = []
            self.grid[cell].append(idx)

    def _point_to_cell(self, point):
        return tuple(np.floor(point / self.cell_size).astype(int))

    def _get_neighboring_cells(self, point, radius):
        cell = self._point_to_cell(point)
        r = int(np.ceil(radius / self.cell_size))
        ranges = [range(c - r, c + r + 1) for c in cell]
        return [tuple(c) for c in np.array(np.meshgrid(*ranges)).T.reshape(-1, 3)]

    def query_ball(self, q, h):
        """Return all point indices within distance h of query point q"""
        neighbors = []
        for cell in self._get_neighboring_cells(q, h):
            if cell in self.grid:
                for idx in self.grid[cell]:
                    if np.linalg.norm(self.points[idx] - q) <= h:
                        neighbors.append(idx)
        return neighbors

    def query_closest(self, q):
        """Return index of the closest point to query q"""
        best_dist = float('inf')
        best_idx = -1
        search_radius = self.cell_size
        found = False

        while not found:
            for cell in self._get_neighboring_cells(q, search_radius):
                if cell in self.grid:
                    for idx in self.grid[cell]:
                        d = np.linalg.norm(self.points[idx] - q)
                        if d < best_dist:
                            best_dist = d
                            best_idx = idx
                            found = True
            search_radius *= 2  # Expand search if nothing found
        return best_idx

if __name__ == "__main__":
    np.random.seed(0)
    points = np.random.rand(1000, 3) * 10  # Random 3D points in [0, 10)^3
    grid = SpatialHashGrid(points, cell_size=1.0)

    test_point = np.array([5.0, 5.0, 5.0])
    h = 1.7

    # Grid-based queries
    grid_neighbors = grid.query_ball(test_point, h)
    grid_closest = grid.query_closest(test_point)

    # Brute-force queries
    brute_neighbors = closest_points(test_point, points,  h)
    brute_closest = find_closest_point(test_point, points)

    # Comparison
    print("Ball query matches:", set(grid_neighbors) == set(brute_neighbors))
    print("Closest point matches:", grid_closest == brute_closest)

Ball query matches: True
Closest point matches: True
